In [1]:
import dgl
import time
import torch as th
import numpy as np
import torch.nn.functional as F

from load_data import load_ogb
from model import SAGE, GAT, get_eigen
from train_eval import train_sample

Using backend: pytorch


This notebook shows the performance of GAT on the OGBN-products graph with scaled eigenvectors as positional embeddings for nodes. The positional node embeddings are fixed in this model. This experiment shows that attention-based models cannot use the positional embeddings better than simple neighbor aggregation like GraphSage.

In [2]:
g, num_labels = load_ogb('ogbn-products')
g.ndata.pop('features')

in_feats = 128
eigen_vals, eigen_vecs = get_eigen(g, in_feats, 'ogbn-products')
train_g = val_g = test_g = g
train_nfeat = val_nfeat = test_nfeat = th.tensor(eigen_vecs * np.sqrt(eigen_vals).reshape((1, len(eigen_vals))),
                                                 dtype=th.float32)
train_labels = val_labels = test_labels = g.ndata.pop('labels')

hyperparams = {
    'batch_size': 1024,
    'num_workers': 0,
    'num_hidden': 256,
    'num_layers': 3,
    'num_heads': 2,
    'dropout': 0.5,
    'lr': 0.003,
    'num_epochs': 20,
    'fanouts': [15,10,5],
    'eval_batch_size': 10000
}
device = th.device('cuda:3')

# Pack data
data = train_g, val_g, test_g, train_nfeat, train_labels, \
           val_nfeat, val_labels, test_nfeat, test_labels
# Define model
model = GAT(in_feats, hyperparams['num_hidden'], num_labels,
             hyperparams['num_layers'], hyperparams['num_heads'], F.relu, hyperparams['dropout'])
train_sample(model, data, hyperparams, device, 3)

load ogbn-products
finish loading ogbn-products
finish constructing ogbn-products


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 3.8491 | Train Acc 0.0371 | Speed (samples/sec) nan | GPU 4325.5 MB
Epoch 00000 | Step 00020 | Loss 1.9916 | Train Acc 0.4600 | Speed (samples/sec) 2123.7115 | GPU 4347.8 MB
Epoch 00000 | Step 00040 | Loss 1.4747 | Train Acc 0.6094 | Speed (samples/sec) 2220.9872 | GPU 4356.9 MB
Epoch 00000 | Step 00060 | Loss 1.2340 | Train Acc 0.6455 | Speed (samples/sec) 2259.4145 | GPU 4356.9 MB
Epoch 00000 | Step 00080 | Loss 1.2125 | Train Acc 0.6582 | Speed (samples/sec) 2299.6371 | GPU 4356.9 MB
Epoch 00000 | Step 00100 | Loss 1.1147 | Train Acc 0.6787 | Speed (samples/sec) 2300.9666 | GPU 4356.9 MB
Epoch 00000 | Step 00120 | Loss 1.0769 | Train Acc 0.6875 | Speed (samples/sec) 2321.6638 | GPU 4356.9 MB
Epoch 00000 | Step 00140 | Loss 1.0054 | Train Acc 0.7227 | Speed (samples/sec) 2371.4029 | GPU 4356.9 MB
Epoch 00000 | Step 00160 | Loss 0.9208 | Train Acc 0.7441 | Speed (samples/sec) 2412.8443 | GPU 4366.7 MB
Epoch 00000 | Step 00180 | Loss 0.8637 | Train Acc 0

100%|██████████| 245/245 [01:36<00:00,  2.55it/s]


Val acc 0.8339, Test Acc: 0.7096
Best val acc 0.8339, best test acc: 0.7096
Epoch 00004 | Step 00000 | Loss 0.6511 | Train Acc 0.8223 | Speed (samples/sec) 2638.1725 | GPU 4384.6 MB
Epoch 00004 | Step 00020 | Loss 0.6482 | Train Acc 0.8271 | Speed (samples/sec) 2640.5045 | GPU 4384.6 MB
Epoch 00004 | Step 00040 | Loss 0.5482 | Train Acc 0.8623 | Speed (samples/sec) 2642.3083 | GPU 4384.6 MB
Epoch 00004 | Step 00060 | Loss 0.6082 | Train Acc 0.8223 | Speed (samples/sec) 2643.2295 | GPU 4384.6 MB
Epoch 00004 | Step 00080 | Loss 0.6474 | Train Acc 0.8242 | Speed (samples/sec) 2643.9707 | GPU 4384.6 MB
Epoch 00004 | Step 00100 | Loss 0.5841 | Train Acc 0.8301 | Speed (samples/sec) 2642.9837 | GPU 4384.6 MB
Epoch 00004 | Step 00120 | Loss 0.6487 | Train Acc 0.8252 | Speed (samples/sec) 2648.9848 | GPU 4384.6 MB
Epoch 00004 | Step 00140 | Loss 0.6365 | Train Acc 0.8291 | Speed (samples/sec) 2648.6746 | GPU 4384.6 MB
Epoch 00004 | Step 00160 | Loss 0.5837 | Train Acc 0.8301 | Speed (samples/s

...

Val acc 0.8622, Test Acc: 0.7284
Best val acc 0.8622, best test acc: 0.7284
Epoch 00013 | Step 00000 | Loss 0.5057 | Train Acc 0.8535 | Speed (samples/sec) 2689.4320 | GPU 4384.8 MB
Epoch 00013 | Step 00020 | Loss 0.5321 | Train Acc 0.8447 | Speed (samples/sec) 2688.0146 | GPU 4384.8 MB
Epoch 00013 | Step 00040 | Loss 0.5114 | Train Acc 0.8506 | Speed (samples/sec) 2686.6290 | GPU 4384.8 MB
Epoch 00013 | Step 00060 | Loss 0.5260 | Train Acc 0.8555 | Speed (samples/sec) 2684.7487 | GPU 4384.8 MB
Epoch 00013 | Step 00080 | Loss 0.5228 | Train Acc 0.8555 | Speed (samples/sec) 2680.2325 | GPU 4384.8 MB
Epoch 00013 | Step 00100 | Loss 0.5128 | Train Acc 0.8467 | Speed (samples/sec) 2678.2336 | GPU 4384.8 MB
Epoch 00013 | Step 00120 | Loss 0.4814 | Train Acc 0.8486 | Speed (samples/sec) 2675.8783 | GPU 4384.8 MB
Epoch 00013 | Step 00140 | Loss 0.5047 | Train Acc 0.8574 | Speed (samples/sec) 2674.1120 | GPU 4384.8 MB
Epoch 00013 | Step 00160 | Loss 0.5197 | Train Acc 0.8496 | Speed (samples/s

100%|██████████| 245/245 [01:42<00:00,  2.39it/s]


Val acc 0.8657, Test Acc: 0.7326
Best val acc 0.8657, best test acc: 0.7326
Epoch 00016 | Step 00000 | Loss 0.5210 | Train Acc 0.8584 | Speed (samples/sec) 2637.3582 | GPU 4384.8 MB
Epoch 00016 | Step 00020 | Loss 0.5073 | Train Acc 0.8535 | Speed (samples/sec) 2637.0436 | GPU 4384.8 MB
Epoch 00016 | Step 00040 | Loss 0.5211 | Train Acc 0.8438 | Speed (samples/sec) 2635.1079 | GPU 4384.8 MB
Epoch 00016 | Step 00060 | Loss 0.3913 | Train Acc 0.8887 | Speed (samples/sec) 2633.9864 | GPU 4384.8 MB
Epoch 00016 | Step 00080 | Loss 0.4725 | Train Acc 0.8750 | Speed (samples/sec) 2633.0194 | GPU 4384.8 MB
Epoch 00016 | Step 00100 | Loss 0.4804 | Train Acc 0.8545 | Speed (samples/sec) 2632.2434 | GPU 4384.8 MB
Epoch 00016 | Step 00120 | Loss 0.4806 | Train Acc 0.8564 | Speed (samples/sec) 2631.6899 | GPU 4384.8 MB
Epoch 00016 | Step 00140 | Loss 0.4811 | Train Acc 0.8555 | Speed (samples/sec) 2630.3185 | GPU 4384.8 MB
Epoch 00016 | Step 00160 | Loss 0.4933 | Train Acc 0.8555 | Speed (samples/s

100%|██████████| 245/245 [01:45<00:00,  2.32it/s]


Val acc 0.7670, Test Acc: 0.6219
Epoch 00019 | Step 00000 | Loss 0.7854 | Train Acc 0.7764 | Speed (samples/sec) 2606.9743 | GPU 4384.8 MB
Epoch 00019 | Step 00020 | Loss 0.5345 | Train Acc 0.8516 | Speed (samples/sec) 2605.5460 | GPU 4384.8 MB
Epoch 00019 | Step 00040 | Loss 0.4878 | Train Acc 0.8555 | Speed (samples/sec) 2604.1199 | GPU 4384.8 MB
Epoch 00019 | Step 00060 | Loss 0.4896 | Train Acc 0.8594 | Speed (samples/sec) 2602.6144 | GPU 4384.8 MB
Epoch 00019 | Step 00080 | Loss 0.4638 | Train Acc 0.8750 | Speed (samples/sec) 2601.5207 | GPU 4384.8 MB
Epoch 00019 | Step 00100 | Loss 0.4131 | Train Acc 0.8721 | Speed (samples/sec) 2600.2060 | GPU 4384.8 MB
Epoch 00019 | Step 00120 | Loss 0.3935 | Train Acc 0.8789 | Speed (samples/sec) 2599.2342 | GPU 4384.8 MB
Epoch 00019 | Step 00140 | Loss 0.4201 | Train Acc 0.8848 | Speed (samples/sec) 2598.3528 | GPU 4384.8 MB
Epoch 00019 | Step 00160 | Loss 0.4748 | Train Acc 0.8623 | Speed (samples/sec) 2597.7374 | GPU 4384.8 MB
Epoch 00019 |